In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np

/usr/local/lib/python3.5/dist-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [3]:
from ensembles import DiversityEnsembleClassifier as DEC
from ensembles import GeneticEnsembleClassifier as GEC
from ensembles import RandomClassifier as RC
from ensembles import MajorityClassifier as MC

### Loading datasets

In [4]:
breast = datasets.load_breast_cancer()
iris   = datasets.load_iris()
wine   = datasets.load_wine()
yeast  = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/yeast/yeast.data', header=None, delim_whitespace=True)
le = LabelEncoder()
yeast.iloc[:, -1] = le.fit_transform(yeast.iloc[:, -1])
#spam   = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data', header=None)
captcha = pd.read_csv('dados_captcha.csv')
captcha.iloc[:, -1] = le.fit_transform(captcha.iloc[:, -1])

In [5]:
captcha.shape

(2788, 78)

<h3>Testing method</h3>

In [11]:
def test_algorithm(classifier, data, target):
    accuracy, f1, precision, recall, auc = 0, 0, 0, 0, 0
    div = np.zeros(25)
    fit = np.zeros(25)
    for i in range(0, 10):
        print(i, end='')
        X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=i*10)
        try:classifier.set_param(random_state=i*10)
        except: classifier.random_state=i*10
        aux = classifier.fit(X_train, y_train)
        try: 
            div += aux[0]
            fit += aux[1]
        except: pass
        y_pred = classifier.predict(X_test)
        accuracy += accuracy_score(y_test, y_pred)
        try: f1 += f1_score(y_test, y_pred)
        except: pass
        try: precision += precision_score(y_test, y_pred)
        except: pass
        try: recall += recall_score(y_test, y_pred)
        except: pass
        try: auc += roc_auc_score(y_test, y_pred)
        except: pass
    print('Accuracy :', accuracy/10)
    if f1>0: print('F1-score :', f1/10)
    if precision>0: print('Precision:', precision/10)
    if recall>0: print('Recall   :', recall/10)
    if auc>0: print('ROC AUC  :', auc/10)
    return div/10, fit/10

In [12]:
def compare_results(data, target, size):
    n_samples = ((data.shape[0]*0.8 * 4) // 5)-1
    alg = {
                KNeighborsClassifier: {'n_neighbors':[1, n_samples]},
                RidgeClassifier: {'alpha':[1.0, 10.0],'max_iter':[10, 100]},
                SVC: {'C':[1, 1000],
                      'gamma':[0.0001, 0.001]
                      },
                DecisionTreeClassifier: {'min_samples_leaf':[1, n_samples], 'max_depth':[1, n_samples]},
                ExtraTreeClassifier: {'min_samples_leaf':[1, n_samples], 'max_depth':[1, n_samples]},
                GaussianNB: {},
                LinearDiscriminantAnalysis: {},
                QuadraticDiscriminantAnalysis: {},
                BernoulliNB: {},
                #GaussianProcessClassifier: {'max_iter_predict':[1, 50]},
                LogisticRegression:{'C':[1, 1000], 'max_iter':[100, 1000]},
                NearestCentroid:{},
                PassiveAggressiveClassifier:{'C':[1, 1000], 'max_iter':[100, 1000]},
                SGDClassifier: {'alpha':[1e-5, 1e-2], 'max_iter':[100, 1000]}
          }
    
    
    print('-'*60)
    print('Diversity-based Ensemble Classifier')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    dec = DEC(alg, population_size=size, max_epochs=25, random_state=42)
    div, fit = test_algorithm(dec, data=data, target=target)
    print('DEC done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')
    
    print('-'*60)
    print('Genetic-based Ensemble Classifier')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    gec = GEC(alg, population_size=size, max_epochs=25, random_state=42)
    test_algorithm(gec, data=data, target=target)
    print('GEC done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')    
                   
    print('-'*60)
    print('Random Classifier')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    rc = RC(alg, random_state=42)
    test_algorithm(rc, data=data, target=target)
    print('RC done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')
      
    print('-'*60)
    print('Majority Classifier')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    mc = MC()
    test_algorithm(mc, data=data, target=target)
    print('MC done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')
    
    print('-'*60)
    print('Random Forest')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    rf = RandomForestClassifier(n_estimators=size, random_state=42)
    test_algorithm(rf, data=data, target=target)
    print('RF done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')
        
    print('-'*60)
    print('Ada Boost')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    ada = AdaBoostClassifier(n_estimators=size, random_state=42)
    test_algorithm(ada, data=data, target=target)
    print('Ada done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')
       
    return div, fit

In [12]:
d10, f10   =compare_results(data=captcha.iloc[:, 1:-1].values, target=captcha.iloc[:, -1].values, size=10)
d25, f25   =compare_results(data=captcha.iloc[:, 1:-1].values, target=captcha.iloc[:, -1].values, size=25)
d50, f50 =compare_results(data=captcha.iloc[:, 1:-1].values, target=captcha.iloc[:, -1].values, size =50)

plt.plot(range(1, 26), d10, label='10 cromossomos')
plt.plot(range(1, 26), d25, label='25 cromossomos')
plt.plot(range(1, 26), d50, label='50 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('captcha.png')
plt.clf()
plt.close()

plt.plot(range(1, 26), f10, label='10 cromossomos')
plt.plot(range(1, 26), f25, label='25 cromossomos')
plt.plot(range(1, 26), f50, label='50 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('captchaFitness.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
0------------------------------------------------------------
Epoch 0
------------------------------------------------------------
Generating offspring...done in 3 ms
Fitting and predicting population...done in 41438 ms
Applying diversity selection...-1
[-inf -inf   0.   0. -inf -inf -inf -inf -inf -inf -inf -inf   0.   0. -inf
 -inf -inf -inf -inf -inf]
0
.1
.2
.3
.done in 3 ms
New population diversity measure: 0.0515445939213
------------------------------------------------------------
Epoch 1
------------------------------------------------------------
Generating offspring...done in 3 ms
Fitting and predicting population...done in 21044 ms
Applying diversity selection...-1
[-inf   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf -inf   0.   0. -inf
 -inf -inf -inf -inf -inf]
0
.1
.2
.3
.4
.5
.6
.7
.8
.done in 4 ms
New population

Fitting and predicting population...done in 1907 ms
Applying diversity selection...-1
[  0.   0.   0.   0. -inf   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.
 -inf   0.   0. -inf   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.done in 4 ms
New population diversity measure: 0.251270553064
------------------------------------------------------------
Epoch 19
------------------------------------------------------------
Generating offspring...done in 3 ms
Fitting and predicting population...done in 2128 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf
   0.   0.   0. -inf   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.done in 4 ms
New population diversity measure: 0.251305430992
------------------------------------------------------------
Epoch 20
------------------------------------------------------------
Generating offspring...done in 4 ms
Fitting and predicting population...done in 719 ms
Applying diversity selection...-1
[  0.   0.   0.   0. -inf

Fitting and predicting population...done in 2435 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0.   0.   0. -inf   0.   0.   0. -inf   0.   0.   0.
   0. -inf   0. -inf -inf]
0
.1
.2
.3
.4
.5
.6
.7
.8
.done in 4 ms
New population diversity measure: 0.251425012456
------------------------------------------------------------
Epoch 38
------------------------------------------------------------
Generating offspring...done in 3 ms
Fitting and predicting population...done in 8485 ms
Applying diversity selection...-1
[  0.   0.   0.   0. -inf   0.   0. -inf   0. -inf   0. -inf   0.   0.   0.
   0.   0.   0. -inf   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.done in 5 ms
New population diversity measure: 0.251425012456
------------------------------------------------------------
Epoch 39
------------------------------------------------------------
Generating offspring...done in 3 ms
Fitting and predicting population...done in 835 ms
Applying diversity selection...-1
[  0.   0.   0.   0. -inf

Fitting and predicting population...done in 1425 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0.   0.   0. -inf -inf   0. -inf   0. -inf -inf   0.
   0.   0.   0.   0.   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.done in 4 ms
New population diversity measure: 0.246028898854
------------------------------------------------------------
Epoch 7
------------------------------------------------------------
Generating offspring...done in 4 ms
Fitting and predicting population...done in 1561 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.
   0. -inf   0. -inf   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.done in 4 ms
New population diversity measure: 0.246357747882
------------------------------------------------------------
Epoch 8
------------------------------------------------------------
Generating offspring...done in 4 ms
Fitting and predicting population...done in 1161 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0. 

Fitting and predicting population...done in 1477 ms
Applying diversity selection...-1
[  0.   0. -inf   0. -inf   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.
   0.   0. -inf -inf   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.done in 4 ms
New population diversity measure: 0.247673143996
------------------------------------------------------------
Epoch 26
------------------------------------------------------------
Generating offspring...done in 3 ms
Fitting and predicting population...done in 1825 ms
Applying diversity selection...-1
[  0.   0. -inf   0. -inf   0.   0.   0.   0. -inf   0.   0.   0.   0. -inf
   0.   0.   0.   0.   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.done in 4 ms
New population diversity measure: 0.247673143996
------------------------------------------------------------
Epoch 27
------------------------------------------------------------
Generating offspring...done in 3 ms
Fitting and predicting population...done in 6366 ms
Applying diversity selection...-1
[  0.   0.   0.   0. -in

...

Fitting and predicting population...done in 3442 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf -inf   0. -inf
   0. -inf -inf   0. -inf]
0
.1
.2
.3
.4
.5
.6
.7
.8
.done in 4 ms
New population diversity measure: 0.250199302441
------------------------------------------------------------
Epoch 37
------------------------------------------------------------
Generating offspring...done in 3 ms
Fitting and predicting population...done in 1381 ms
Applying diversity selection...-1
[  0.   0.   0. -inf   0.   0.   0.   0.   0.   0.   0. -inf   0.   0. -inf
   0.   0.   0.   0. -inf]
0
.1
.2
.3
.4
.5
.6
.7
.8
.done in 4 ms
New population diversity measure: 0.250199302441
------------------------------------------------------------
Epoch 38
------------------------------------------------------------
Generating offspring...done in 3 ms
Fitting and predicting population...done in 10848 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   

Fitting and predicting population...done in 12040 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0.   0.   0. -inf -inf   0.   0.   0. -inf   0.   0.
   0. -inf   0.   0.   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.done in 4 ms
New population diversity measure: 0.251305430992
------------------------------------------------------------
Epoch 6
------------------------------------------------------------
Generating offspring...done in 4 ms
Fitting and predicting population...done in 511 ms
Applying diversity selection...-1
[  0.   0. -inf   0.   0. -inf   0. -inf -inf   0.   0. -inf -inf   0.   0.
   0. -inf   0.   0. -inf]
0
.1
.2
.3
.4
.5
.6
.7
.8
.done in 3 ms
New population diversity measure: 0.251415047334
------------------------------------------------------------
Epoch 7
------------------------------------------------------------
Generating offspring...done in 4 ms
Fitting and predicting population...done in 1625 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0. 

Fitting and predicting population...done in 4328 ms
Applying diversity selection...-1
[  0.   0. -inf   0. -inf   0. -inf -inf   0.   0.   0.   0.   0.   0. -inf
   0.   0.   0.   0.   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.done in 4 ms
New population diversity measure: 0.252670652715
------------------------------------------------------------
Epoch 25
------------------------------------------------------------
Generating offspring...done in 3 ms
Fitting and predicting population...done in 4560 ms
Applying diversity selection...-1
[  0.   0. -inf   0.   0.   0.   0.   0.   0. -inf   0. -inf -inf   0.   0.
   0.   0.   0.   0.   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.done in 5 ms
New population diversity measure: 0.252670652715
------------------------------------------------------------
Epoch 26
------------------------------------------------------------
Generating offspring...done in 3 ms
Fitting and predicting population...done in 643 ms
Applying diversity selection...-1
[  0.   0.   0.   0. -inf

Fitting and predicting population...done in 10328 ms
Applying diversity selection...-1
[  0.   0.   0. -inf -inf   0.   0.   0.   0. -inf   0. -inf   0. -inf   0.
   0.   0. -inf   0.   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.done in 4 ms
New population diversity measure: 0.252730443448
------------------------------------------------------------
Epoch 44
------------------------------------------------------------
Generating offspring...done in 3 ms
Fitting and predicting population...done in 2629 ms
Applying diversity selection...-1
[  0.   0.   0.   0. -inf   0.   0.   0.   0. -inf   0. -inf   0.   0. -inf
   0.   0.   0.   0.   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.done in 4 ms
New population diversity measure: 0.252770303936
------------------------------------------------------------
Epoch 45
------------------------------------------------------------
Generating offspring...done in 3 ms
Fitting and predicting population...done in 7001 ms
Applying diversity selection...-1
[  0.   0.   0.   0. -i

TypeError: fit() takes 2 positional arguments but 3 were given

In [8]:
d25, f25   =compare_results(data=breast.data, target=breast.target, size=25)
d50, f50   =compare_results(data=breast.data, target=breast.target, size=50)
d100, f100 =compare_results(data=breast.data, target=breast.target, size =100)

plt.plot(range(1, 101), d25, label='25 cromossomos')
plt.plot(range(1, 101), d50, label='50 cromossomos')
plt.plot(range(1, 101), d100, label='100 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('breast2.png')
plt.clf()
plt.close()

plt.plot(range(1, 101), f25, label='25 cromossomos')
plt.plot(range(1, 101), f50, label='50 cromossomos')
plt.plot(range(1, 101), f100, label='100 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('breastFitness.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
0------------------------------------------------------------
Epoch 0
------------------------------------------------------------
Generating offspring...done in 6 ms
Fitting and predicting population...done in 1199 ms
Applying diversity selection...-1
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 8 ms
New population diversity measure: 0.217571428571
------------------------------------------------------------
Epoch 1
------------------------------------------------------------
Generating offspring...done in 7 ms
Fitting and predicting population...done in 467 ms
Applying diversity s

Fitting and predicting population...done in 538 ms
Applying diversity selection...-1
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 8 ms
New population diversity measure: 0.355666666667
------------------------------------------------------------
Epoch 14
------------------------------------------------------------
Generating offspring...done in 7 ms
Fitting and predicting population...done in 440 ms
Applying diversity selection...-1
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 8 ms
New popu

Fitting and predicting population...done in 500 ms
Applying diversity selection...-1
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 8 ms
New population diversity measure: 0.384424908425
------------------------------------------------------------
Epoch 28
------------------------------------------------------------
Generating offspring...done in 7 ms
Fitting and predicting population...done in 223 ms
Applying diversity selection...-1
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 8 ms
New popu

Fitting and predicting population...done in 314 ms
Applying diversity selection...-1
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 7 ms
New population diversity measure: 0.417128205128
------------------------------------------------------------
Epoch 42
------------------------------------------------------------
Generating offspring...done in 8 ms
Fitting and predicting population...done in 383 ms
Applying diversity selection...-1
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 9 ms
New popu

Fitting and predicting population...done in 353 ms
Applying diversity selection...-1
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 9 ms
New population diversity measure: 0.388399267399
------------------------------------------------------------
Epoch 6
------------------------------------------------------------
Generating offspring...done in 7 ms
Fitting and predicting population...done in 248 ms
Applying diversity selection...-1
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 8 ms
New popul

...

Fitting and predicting population...done in 759 ms
Applying diversity selection...-1
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.24
.25
.26
.27
.28
.29
.30
.31
.32
.33
.34
.35
.36
.37
.38
.39
.40
.41
.42
.43
.44
.45
.46
.47
.48
.done in 18 ms
New population diversity measure: 0.363662704642
------------------------------------------------------------
Epoch 19
------------------------------------------------------------
Generating offspring...done in 14 ms
Fitting and predicting population...done in 953 ms
Applying diversity selection...-1
[ 0.  

Fitting and predicting population...done in 938 ms
Applying diversity selection...-1
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.24
.25
.26
.27
.28
.29
.30
.31
.32
.33
.34
.35
.36
.37
.38
.39
.40
.41
.42
.43
.44
.45
.46
.47
.48
.done in 17 ms
New population diversity measure: 0.385437990581
------------------------------------------------------------
Epoch 28
------------------------------------------------------------
Generating offspring...done in 15 ms
Fitting and predicting population...done in 757 ms
Applying diversity selection...-1
[ 0.  

Fitting and predicting population...done in 817 ms
Applying diversity selection...-1
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.24
.25
.26
.27
.28
.29
.30
.31
.32
.33
.34
.35
.36
.37
.38
.39
.40
.41
.42
.43
.44
.45
.46
.47
.48
.done in 18 ms
New population diversity measure: 0.399786499215
------------------------------------------------------------
Epoch 37
------------------------------------------------------------
Generating offspring...done in 16 ms
Fitting and predicting population...done in 743 ms
Applying diversity selection...-1
[ 0.  

Fitting and predicting population...done in 655 ms
Applying diversity selection...-1
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.24
.25
.26
.27
.28
.29
.30
.31
.32
.33
.34
.35
.36
.37
.38
.39
.40
.41
.42
.43
.44
.45
.46
.47
.48
.done in 20 ms
New population diversity measure: 0.414535994618
------------------------------------------------------------
Epoch 46
------------------------------------------------------------
Generating offspring...done in 15 ms
Fitting and predicting population...done in 1207 ms
Applying diversity selection...-1
[ 0. 

KeyboardInterrupt: 

In [ ]:
d25, f25 = compare_results(data=iris.data, target=iris.target, size=25)

d50, f50 = compare_results(data=iris.data, target=iris.target, size=50)

d100, f100 = compare_results(data=iris.data, target=iris.target, size=100)

plt.plot(range(1, 26), d25, label='25 cromossomos')
plt.plot(range(1, 26), d50, label='50 cromossomos')
plt.plot(range(1, 26), d100, label='100 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('iris.png')
plt.clf()
plt.close()


plt.plot(range(1, 26), f25, label='25 cromossomos')
plt.plot(range(1, 26), f50, label='50 cromossomos')
plt.plot(range(1, 26), f100, label='100 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('irisFitness.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
0------------------------------------------------------------
Epoch 0
------------------------------------------------------------
Generating offspring...done in 9 ms
Fitting and predicting population...done in 318 ms
Applying diversity selection...-1
[  0.   0. -inf   0. -inf   0.   0.   0. -inf   0. -inf   0.   0.   0.   0.
 -inf   0.   0.   0.   0.   0. -inf   0.   0.   0.   0. -inf -inf   0.   0.
 -inf   0.   0. -inf   0.   0.   0. -inf   0.   0. -inf -inf   0.   0. -inf
   0.   0.   0.   0.   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 7 ms
New population diversity measure: 0.12968
------------------------------------------------------------
Epoch 1
------------------------------------------------------------
Generating offspring...done in 7 ms
Fitting and predicting popul

Fitting and predicting population...done in 196 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0.   0. -inf -inf   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0. -inf   0. -inf   0.   0.   0.   0.   0.   0.   0. -inf
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.
   0.   0.   0.   0.   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 8 ms
New population diversity measure: 0.374386666667
------------------------------------------------------------
Epoch 14
------------------------------------------------------------
Generating offspring...done in 7 ms
Fitting and predicting population...done in 166 ms
Applying diversity selection...-1
[  0.   0.   0. -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. -inf   0. -inf   0. -inf -inf   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.]
0
.1


Fitting and predicting population...done in 107 ms
Applying diversity selection...-1
[  0.   0. -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.
   0.   0. -inf   0.   0. -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0. -inf   0.   0.   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 8 ms
New population diversity measure: 0.34864
------------------------------------------------------------
Epoch 3
------------------------------------------------------------
Generating offspring...done in 8 ms
Fitting and predicting population...done in 172 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. -inf   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.
   0. -inf   0.   0.   0.]
0
.1
.2
.3
.4

Fitting and predicting population...done in 168 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 7 ms
New population diversity measure: 0.396946666667
------------------------------------------------------------
Epoch 17
------------------------------------------------------------
Generating offspring...done in 7 ms
Fitting and predicting population...done in 137 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0.   0. -inf   0.   0.   0. -inf   0.   0.   0.   0.
 -inf   0.   0. -inf   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.   0.
   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.]
0
.1


Fitting and predicting population...done in 180 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.
 -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 7 ms
New population diversity measure: 0.36428
------------------------------------------------------------
Epoch 5
------------------------------------------------------------
Generating offspring...done in 7 ms
Fitting and predicting population...done in 165 ms
Applying diversity selection...-1
[  0.   0. -inf -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf
   0.   0.   0.   0. -inf -inf   0.   0.   0. -inf   0.   0.   0.   0.   0.
   0. -inf   0. -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. -inf   0.   0.]
0
.1
.2
.3
.4

...

Fitting and predicting population...done in 592 ms
Applying diversity selection...-1
[  0.   0.   0.   0. -inf   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.   0.   0. -inf   0.
   0.   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.   0.   0.   0.
   

Fitting and predicting population...done in 567 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.   0. -inf   0. -inf
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0. -inf   0. -inf -inf   0.   0.   0. -inf   0.   0.   0. -inf   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0. -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.
   0. -inf   0.   0.   0. -inf -inf -inf -inf   0.   0.   0.   0. -inf   0.
 -inf   0.   0. -inf -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   

Fitting and predicting population...done in 483 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 -inf   0.   0.   0.   0.   0.   0.   0. -inf   0. -inf   0. -inf   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.   0. -inf   0.
   0.   0.   0.   0.   0.   0. -inf   0. -inf   0.   0.   0.   0.   0.   0.
   0.   0. -inf   0. -inf   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.
   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.   0.   0.
   

Fitting and predicting population...done in 733 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0. -inf   0.   0.   0. -inf   0.   0.   0.   0.   0.
   0.   0.   0.   0. -inf   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.
   0. -inf   0. -inf -inf   0.   0.   0.   0.   0. -inf   0.   0. -inf   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.
   0.   0.   0. -inf   0.   0.   0.   0.   0.   0. -inf   0.   0.   0. -inf
   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.   0.   0.
   0.   0. -inf -inf   0. -inf   0.   0.   0.   0.   0. -inf   0.   0.   0.
 -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.
   0.   0. -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0. -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   

0123456789

In [9]:
d25, f25 = compare_results(data=wine.data, target=wine.target, size=25)
d50, f50 = compare_results(data=wine.data, target=wine.target, size=50)
d100, f100 = compare_results(data=wine.data, target=wine.target, size=100)

plt.plot(range(1, 51), d25, label='25 cromossomos')
plt.plot(range(1, 51), d50, label='50 cromossomos')
plt.plot(range(1, 51), d100, label='100 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('wine.png')
plt.clf()
plt.close()

plt.plot(range(1, 51), f25, label='25 cromossomos')
plt.plot(range(1, 51), f50, label='50 cromossomos')
plt.plot(range(1, 51), f100, label='100 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('wineFitness.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
Accuracy : 0.8666666666666668
DEC done in:  20594.8 ms
------------------------------------------------------------
Genetic-based Ensemble Classifier
------------------------------------------------------------
Accuracy : 0.986111111111111
GEC done in:  19775.3 ms
------------------------------------------------------------
Random Classifier
------------------------------------------------------------
Accuracy : 0.31111111111111106
RC done in:  0.9 ms
------------------------------------------------------------
Random Forest
------------------------------------------------------------
Accuracy : 0.9694444444444443
RF done in:  38.6 ms
------------------------------------------------------------
Ada Boost
------------------------------------------------------------
Accuracy : 0.8777777777777777
Ada done in:  41.9 ms
---------------

KeyboardInterrupt: 

In [11]:
d25 = compare_results(data=yeast.iloc[:, 1:-1].values, target=yeast.iloc[:, -1].values, size = 25)
d50 = compare_results(data=yeast.iloc[:, 1:-1].values, target=yeast.iloc[:, -1].values, size = 50)
d100 = compare_results(data=yeast.iloc[:, 1:-1].values, target=yeast.iloc[:, -1].values, size = 100)

plt.plot(range(1, 101), d25, label='25 cromossomos')
plt.plot(range(1, 101), d50, label='50 cromossomos')
plt.plot(range(1, 101), d100, label='100 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('yeast.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
0------------------------------------------------------------
Epoch 0
------------------------------------------------------------
Generating offspring...done in 9 ms
Fitting and predicting population...done in 14530 ms
Applying diversity selection...-1
[-inf   0. -inf -inf   0. -inf   0.   0. -inf   0.   0.   0.   0. -inf   0.
   0. -inf   0.   0.   0.   0. -inf -inf -inf -inf   0.   0. -inf -inf -inf
   0.   0. -inf   0.   0. -inf -inf   0. -inf   0.   0. -inf   0.   0.   0.
   0. -inf -inf -inf -inf]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 10 ms
New population diversity measure: 0.195839651783
------------------------------------------------------------
Epoch 1
------------------------------------------------------------
Generating offspring...done in 7 ms
Fitting and predic

Fitting and predicting population...done in 2374 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf -inf   0.
   0.   0. -inf   0.   0.   0.   0. -inf -inf -inf   0.   0.   0.   0.   0.
   0. -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.
 -inf   0.   0.   0.   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 11 ms
New population diversity measure: 0.274289525414
------------------------------------------------------------
Epoch 13
------------------------------------------------------------
Generating offspring...done in 7 ms
Fitting and predicting population...done in 3524 ms
Applying diversity selection...-1
[  0.   0.   0.   0. -inf   0. -inf   0.   0.   0.   0. -inf   0.   0.   0.
   0.   0. -inf -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0. -inf   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.   0.   0.
   0. -inf   0.   0. -inf]
0


Fitting and predicting population...done in 6065 ms
Applying diversity selection...-1
[-inf   0.   0.   0.   0.   0.   0.   0.   0. -inf   0.   0.   0.   0.   0.
   0. -inf   0.   0. -inf   0.   0.   0.   0.   0. -inf   0.   0. -inf   0.
   0.   0. -inf   0.   0. -inf -inf   0.   0.   0.   0.   0.   0.   0.   0.
   0. -inf -inf   0.   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 10 ms
New population diversity measure: 0.278535523729
------------------------------------------------------------
Epoch 26
------------------------------------------------------------
Generating offspring...done in 8 ms
Fitting and predicting population...done in 7598 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0.   0.   0.   0.   0. -inf -inf   0.   0.   0.   0.
   0. -inf -inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf
   0.   0.   0.   0.   0. -inf -inf   0.   0.   0.   0.   0.   0.   0.   0.
   0. -inf   0. -inf   0.]
0


Fitting and predicting population...done in 5259 ms
Applying diversity selection...-1
[-inf   0.   0.   0.   0.   0. -inf -inf   0. -inf   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -inf   0. -inf -inf   0.
   0.   0.   0. -inf -inf -inf   0.   0.   0. -inf   0.   0.   0.   0.   0.
   0. -inf   0.   0.   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 10 ms
New population diversity measure: 0.281647009267
------------------------------------------------------------
Epoch 39
------------------------------------------------------------
Generating offspring...done in 8 ms
Fitting and predicting population...done in 9047 ms
Applying diversity selection...-1
[-inf   0.   0.   0.   0.   0. -inf   0.   0. -inf   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0. -inf   0.   0. -inf -inf   0.   0.   0.   0.   0.
   0.   0. -inf -inf   0. -inf   0.   0.   0.   0.   0.   0.   0. -inf   0.
   0.   0.   0.   0. -inf]
0


/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:710: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:710: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:713: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:710: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:710: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:713: RuntimeWarning: divide by zero encountered in log
  u = np.asarray

...

/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:710: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:710: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:713: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])


done in 23461 ms
Applying diversity selection...-1
[  0. -inf   0. -inf -inf -inf -inf -inf -inf   0. -inf   0. -inf   0. -inf
 -inf -inf -inf -inf -inf   0. -inf -inf   0. -inf   0. -inf   0. -inf -inf
 -inf -inf -inf -inf -inf -inf -inf   0. -inf -inf -inf -inf -inf -inf   0.
   0. -inf   0.   0.   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.done in 7 ms
New population diversity measure: 0.14966769634
------------------------------------------------------------
Epoch 1
------------------------------------------------------------
Generating offspring...done in 8 ms
Fitting and predicting population...

/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:710: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:710: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:713: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])


done in 10851 ms
Applying diversity selection...-1
[  0.   0.   0.   0.   0.   0. -inf   0. -inf   0.   0.   0.   0.   0.   0.
   0.   0. -inf   0.   0.   0. -inf -inf   0.   0.   0.   0.   0.   0. -inf
 -inf -inf   0. -inf -inf -inf -inf   0.   0. -inf -inf -inf -inf -inf   0.
   0. -inf   0.   0.   0.]
0
.1
.2
.3
.4
.5
.6
.7
.8
.9
.10
.11
.12
.13
.14
.15
.16
.17
.18
.19
.20
.21
.22
.23
.done in 10 ms
New population diversity measure: 0.264077506318
------------------------------------------------------------
Epoch 2
------------------------------------------------------------
Generating offspring...done in 7 ms
Fitting and predicting population...

KeyboardInterrupt: 

In [ ]:
d25 = compare_results(data=spam.iloc[:, :-1].values, target=spam.iloc[:, -1].values, size=25)
d50 = compare_results(data=spam.iloc[:, :-1].values, target=spam.iloc[:, -1].values, size=50)
d100 = compare_results(data=spam.iloc[:, :-1].values, target=spam.iloc[:, -1].values, size=100)

plt.plot(range(1, 101), d25, label='25 cromossomos')
plt.plot(range(1, 101), d50, label='50 cromossomos')
plt.plot(range(1, 101), d100, label='100 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('spam.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
Accuracy : 0.7684039087947883
F1-score : 0.5886980455891058
Precision: 0.8610153167004378
Recall   : 0.4860016059630662
ROC AUC  : 0.7170294642575683
DEC done in:  9712588.9 ms
------------------------------------------------------------
Genetic-based Ensemble Classifier
------------------------------------------------------------
